# Processing Metadata

**Methodology**:
1. Extract and format maize management features for all year-locations (aka envs)
2. Clean file by imputing missing values where possible, otherwise filling in missing values with 'n/a' string
3. Label encode and/or normalise values for all extracted features
4. Construct and evaluate random forest model predicting corn yield with snp, weather and management features

In [1]:
import pandas as pd
print(pd.__version__)

2.2.3


In [ ]:
metadata = pd.read_csv('2_Training_Meta_Data_2014_2023.csv')
metadata.head(10)

,Year,Env,Experiment_Code,Treatment,City,Farm,Field,Trial_ID (Assigned by collaborator for internal reference),"Soil_Taxonomic_ID and horizon description, if known","Weather_Station_Serial_Number (Last four digits, e.g. m2700s#####)",...,Cardinal_Heading_Pass_1,Irrigated,Issue/comment_#1,Issue/comment_#2,Issue/comment_#3,Issue/comment_#4,Issue/comment_#5,Issue/comment_#6,Issue/comment_#7,Issue/comment_#8
0,2014,DEH1_2014,DEH1,NaN,Georgetown,Elbert N. & Ann V. Carvel Research & Education...,27AB,NaN,NaN,9079,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014,GAH1_2014,GAH1,NaN,Tifton,USDA - Bellflower experimental farm,18,NaN,NaN,8427,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014,IAH1a_2014,IAH1,NaN,Ames,Worle,NaN,NaN,NaN,9080,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Information for IAH1a_2014, IAH1b_2014,and IAH...",NaN
3,2014,IAH1b_2014,IAH1,NaN,Ames,Worle,NaN,NaN,NaN,9080,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Information for IAH1a_2014, IAH1b_2014,and IAH...",NaN
4,2014,IAH1c_2014,IAH1,NaN,Ames,Worle,NaN,NaN,NaN,9080,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Information for IAH1a_2014, IAH1b_2014,and IAH...",NaN
5,2014,IAH2_2014,IAH2,NaN,Carroll,NaN,NaN,NaN,NaN,9083,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2014,IAH3_2014,IAH3,NaN,Keystone,NaN,NaN,NaN,NaN,9085,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2014,IAH4_2014,IAH4,NaN,Crawfordsville,Southeast Research Farm,14,NaN,NaN,9082,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2014,ILH1_2014,ILH1,NaN,Urbana,Maxwell Farms,MF500,NaN,NaN,8653,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2014,INH1_2014,INH1,NaN,West Lafayette,Purdue ACRE,97/98,NaN,NaN,8657,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
len(metadata)
# metadata info available for 272 year-locations

272

In [ ]:
metadata.columns

Index(['Year', 'Env', 'Experiment_Code', 'Treatment', 'City', 'Farm', 'Field',
       'Trial_ID (Assigned by collaborator for internal reference)',
       'Soil_Taxonomic_ID and horizon description, if known',
       'Weather_Station_Serial_Number (Last four digits, e.g. m2700s#####)',
       'Weather_Station_Latitude (in decimal numbers NOT DMS)',
       'Weather_Station_Longitude (in decimal numbers NOT DMS)',
       'Date_weather_station_placed', 'Date_weather_station_removed',
       'Previous_Crop', 'Pre-plant_tillage_method(s)',
       'In-season_tillage_method(s)',
       'Type_of_planter (fluted cone; belt cone; air planter)',
       'System_Determining_Moisture', 'Pounds_Needed_Soil_Moisture',
       'Latitude_of_Field_Corner_#1 (lower left)',
       'Longitude_of_Field_Corner_#1 (lower left)',
       'Latitude_of_Field_Corner_#2 (lower right)',
       'Longitude_of_Field_Corner_#2 (lower right)',
       'Latitude_of_Field_Corner_#3 (upper right)',
       'Longitude_of_Field_C

In [ ]:
#Calculate the average latitude of the four field corners and store it in a new 'lat' column
metadata["lat"] = metadata[['Latitude_of_Field_Corner_#1 (lower left)', 'Latitude_of_Field_Corner_#2 (lower right)',
             'Latitude_of_Field_Corner_#3 (upper right)','Latitude_of_Field_Corner_#4 (upper left)',]].mean(axis=1)

#Calculate the average longitude of the four field corners and store it in a new 'lon' column
metadata["lon"] = metadata[['Longitude_of_Field_Corner_#1 (lower left)', 'Longitude_of_Field_Corner_#2 (lower right)',
            'Longitude_of_Field_Corner_#3 (upper right)','Longitude_of_Field_Corner_#4 (upper left)',]].mean(axis=1)


In [ ]:
import re

comments = metadata[['Issue/comment_#1', 'Issue/comment_#2', 'Issue/comment_#3',
                     'Issue/comment_#4', 'Issue/comment_#5', 'Issue/comment_#6',
                     'Issue/comment_#7', 'Issue/comment_#8']].copy()

def replace_keywords(text):
    """This function checks whether a given string matches any words defined in a predefined dictionary.

  Function will:
  If there is a match, the string is either replaced with a simplified category label (e.g., 'water', 'animal') or
  If the match is from the 'keywords' category, the actual matching word is retained.
  All non-matching strings and missing values are replaced with 'None' ."""

    # Define keywords and their categories
    keyword_categories = {
        "keywords": ["weed", "lodg", "feed", "stunt", "germination", "stand", "emergence", "rot"],
        "wind": ["wind", "gooseneck", "goose neck", "greensnap", "hail"],
        "weed" :["morning glory", "pigweed", "watchdog", "herbicide"],
        "water": ["drought", "dry", "flood", "rain", "moisture", "water", "underwater", "irrigation", "irrigated"],
        "equip": ["break", "battery", "jam", "disk", "cultivator", "weather", 
                  "sensor", "wire", "additional weather", "tractor", "roller", 
                  "plugged", "sprayer", "planter", "combine", "crew", "mistake", "miscommunication"],
        "animal": ["raccoon", "goose", "deer", "rodents", "hog"]
    }
    
    if isinstance(text, str):  
        text_lower = text.lower() 
        
        # Match each category with its radicals
        for category, radicals in keyword_categories.items():
            for radical in radicals:
                if re.search(rf"\b{radical}", text_lower): 
                    if category == "keywords":
                        return radical  
                    elif category == "wind":
                        return "wind damage"
                    elif category == "water":
                        return "water"
                    elif category == "weed":
                        return "weed"
                    elif category == "equip":
                        return "equipment"
                    elif category == "animal":
                        return "animal"
    return None  

In [ ]:
#apply function
for col in comments.columns :
    print(f"Processing column: {col}")
    comments[col] = comments[col].apply(replace_keywords) 

Processing column: Issue/comment_#1
Processing column: Issue/comment_#2
Processing column: Issue/comment_#3
Processing column: Issue/comment_#4
Processing column: Issue/comment_#5
Processing column: Issue/comment_#6
Processing column: Issue/comment_#7
Processing column: Issue/comment_#8


In [ ]:
unique_values = comments.stack().unique()
print(len(unique_values))
print(unique_values)

12
['weed' 'feed' 'lodg' 'equipment' 'stunt' 'water' 'stand' 'rot' 'animal'
 'germination' 'wind damage' 'emergence']


In [ ]:
# Multiple comments across columns are combined
metadata['Issues'] = comments.apply(lambda row: ', '.join(filter(None, row)), axis=1)

In [ ]:
# Extract key raw and processed management features
metadata = metadata[['Year', 'Env', 'Experiment_Code', 'Treatment',
       'Soil_Taxonomic_ID and horizon description, if known',
       'Previous_Crop', 'Pre-plant_tillage_method(s)',
       'In-season_tillage_method(s)', 'Issues', 'lon', 'lat',
       'Type_of_planter (fluted cone; belt cone; air planter)', 'Irrigated']]



In [136]:
metadata.head(5)

,Year,Env,Experiment_Code,Treatment,"Soil_Taxonomic_ID and horizon description, if known",Previous_Crop,Pre-plant_tillage_method(s),In-season_tillage_method(s),Issues,lon,lat,Type_of_planter (fluted cone; belt cone; air planter),Irrigated
0,2014,DEH1_2014,DEH1,NaN,NaN,soybean,Conventional,NaN,NaN,NaN,NaN,Air planter,NaN
1,2014,GAH1_2014,GAH1,NaN,NaN,cotton,conventional,NaN,NaN,NaN,NaN,fluted cone,NaN
2,2014,IAH1a_2014,IAH1,NaN,NaN,soybean,field cultivator,NaN,NaN,NaN,NaN,Air planter,NaN
3,2014,IAH1b_2014,IAH1,NaN,NaN,soybean,field cultivator,NaN,NaN,NaN,NaN,Air planter,NaN
4,2014,IAH1c_2014,IAH1,NaN,NaN,soybean,field cultivator,NaN,NaN,NaN,NaN,Air planter,NaN


In [ ]:
missing_values = metadata.isnull().sum()
print(f"Missing values per column:\n{missing_values}")

Missing values per column:
Year                                                       0
Env                                                        0
Experiment_Code                                            0
Treatment                                                 56
Soil_Taxonomic_ID and horizon description, if known      195
Previous_Crop                                             34
Pre-plant_tillage_method(s)                               57
In-season_tillage_method(s)                              186
Issues                                                   178
lon                                                       52
lat                                                       52
Type_of_planter (fluted cone; belt cone; air planter)     25
Irrigated                                                222
dtype: int64


In [29]:
#col: Treatment (56/272 missing values)

print(metadata['Treatment'].value_counts(dropna=False))
#Late Planting = Late planting 
#Dryland = Dry Land
#Assume NaN is different to Standard

Treatment
Standard                        192
NaN                              56
Drought                           8
Disease trial                     4
Dry Land                          2
Late Planting                     2
Early Planting                    1
Irrigated                         1
Late Stressed                     1
Dryland                           1
Standard - Irrigated Optimal      1
Dryland optimal                   1
Late Planted Irrigated            1
Late planting                     1
Name: count, dtype: int64


In [30]:
#replace Late planting string as Late Planting string 
metadata['Treatment'] = metadata['Treatment'].replace('Late planting', 'Late Planting')

#replace Dryland string with Dry Land string
metadata['Treatment'] = metadata['Treatment'].replace('Dryland', 'Dry Land')

In [31]:
#col: Soil_Taxonomic_ID and horizon description, if known (195/272 missing values)

print(metadata['Soil_Taxonomic_ID and horizon description, if known'].unique())
#49 unique categories (excluding nan)
#many instance will not have a Soil_Taxanomic_ID (could be an area of data bias, regions is soil id could be grouped together better)

[nan 'Tifton loamy sand/Dothan loamy sand' 'Augusta A / Cartecey A'
 'Flanagan Silt Loam (154A)' '\tNicollet clay loam'
 '5058  Mexico Silt Loam, 0-2% slopes'
 'Lynchburg sandy loam, 0 to 2 percent slopes, Ap - 0 to 8 inches: sandy loam'
 'flat' 'Tifton loamy sand' 'Masada A/Wickham sandy loam' 'capac loam'
 '5058 Mexico Silt Loam, 0-2% slopes' 'Pantego loam' 'Alfisol'
 '2w3kh & 2trwv' 'LtA' 'Flanagan Silt Loam' 'RcA-Raub-Brenton Complex'
 'Silt loam' '5058/5059ÊMexico Silt Loam' '5059 Mexico Silt Loam'
 'silt/clay/loam' 'Lima silt loam' 'KlA/LtB' 'Raub-Brenton complex'
 'Webster Clay Loam' 'Mexico Silt Loam, 1-4 percent slopes, eroded'
 'Oxyaquic Hapludalfs' 'HnB/LtA' 'LtA/HnB' 'PoA, TrB' 'PnA, PnB'
 'Mexico Silt Loam, 1-4 percent slopes, eroded. https://soilseries.sc.egov.usda.gov/OSD_Docs/M/MEXICO.html'
 'Lynchburg sandy loam, 0 to 2 percent slopes:            Ap - 0 to 8 inches: sandy loam E - 8 to 11 inches: sandy loam Bt - 11 to 21 inches: sandy clay loam Btg - 21 to 65 inches: s

In [32]:
#col: previous crop (34/272 missing values)

print(metadata['Previous_Crop'].value_counts(dropna=False))
#soybean and Soybeans same
#sorghum and Sorghum same
#Winter wheat and Winter Wheat same
#wheat/double crop soybean and wheat and Double Crop soybean and wheat/soybean are same
#many of the previous crop is soybean

Previous_Crop
soybean                                                                                                                               159
NaN                                                                                                                                    34
corn                                                                                                                                   30
wheat                                                                                                                                  16
peanut                                                                                                                                  6
Winter wheat                                                                                                                            4
sorghum                                                                                                                                 3
Sorghum             

In [33]:
#uniform upper and lower case formating of same crop
metadata['Previous_Crop'] = metadata['Previous_Crop'].replace('soybean', 'Soybeans')
metadata['Previous_Crop'] = metadata['Previous_Crop'].replace('sorghum', 'Sorghum')
metadata['Previous_Crop'] = metadata['Previous_Crop'].replace('Winter wheat', 'Winter Wheat')

#combine groups with wheat and soybean as a double crop
metadata['Previous_Crop'] = metadata['Previous_Crop'].replace('wheat/double crop soybean', 'wheat/soybean')
metadata['Previous_Crop'] = metadata['Previous_Crop'].replace('wheat and Double Crop soybean', 'wheat/soybean')

In [34]:
#col: Pre plant tillage method  (57/272 missing values)
metadata['Pre-plant_tillage_method(s)'].unique().tolist()

['Conventional',
 'conventional',
 'field cultivator',
 'No-till',
 'Chisel plow and field cultivator',
 'Fall chisel plow and spring field cultivate',
 'chisel plow in fall; field cultivated in spring',
 'Disc in previous fall',
 'In the Spring the land was cut with a disk, then ripped with a chisel plow to a depth of 8-10”. It was then cut again and we applied 300#/acre of 10-0-30-12%S. Next we used a field cultivator with rolling baskets to incorporate the fertilizer. The land was bedded just prior to planting.',
 'chisel',
 'no-till',
 'Field J was fall moldboard plow;  Then disked this spring and field cultivated before planting.',
 'The field was minium tilled.  The field was disked then cultipacked then Cultimulched then planted',
 'Fall Chisel Plow; Spring Cultivate',
 'min-till',
 nan,
 'Fall Chisel',
 'Field cultivator',
 'Field cultivate',
 'fall chisel plow, spring field cultivator',
 'disc, conventional, followed by bedding',
 'No-Till',
 'No Till',
 'Chisel plowed 5/4/15 

In [35]:
#convert all strings to lower case to group together same tillage methods with different upper and lower case formating
metadata['Pre-plant_tillage_method(s)'] = metadata['Pre-plant_tillage_method(s)'].str.lower()

In [36]:
#change spelling of disc to disk (alternate spelling)
metadata['Pre-plant_tillage_method(s)'] = metadata['Pre-plant_tillage_method(s)'].replace('disc', 'disk')

#minimal editing of values.
#for example, Manual weeding could be hand weed but went unaltered.

In [37]:
#col: In season tillage methods (186/272 missing values)
print(metadata['In-season_tillage_method(s)'].value_counts(dropna=False))

In-season_tillage_method(s)
NaN                                                         186
none                                                         44
Cultivator                                                   10
Hand hoeing                                                   4
CaseIH VT 360 vertical tillage tool gone over 2X on 5/30      3
Cultivator tilling                                            3
hand weed                                                     2
disc                                                          2
Disked on 05/17/18 AM, Rolling Harrow on 05/17/18 PM          2
cultivate                                                     2
disk                                                          1
cultivation                                                   1
plow                                                          1
Disked, chisel plow & final disking                           1
Disced and cultimulched                                       1
Field Cultiv

In [38]:
#convert all strings to lower case to group together same tillage methods with different upper and lower case formating
metadata['In-season_tillage_method(s)'] = metadata['In-season_tillage_method(s)'].str.lower()

In [39]:
#change disc to disk
metadata['In-season_tillage_method(s)'] = metadata['In-season_tillage_method(s)'].replace('disc', 'disk')

#minimal editing of values

In [40]:
#col: Issues (177/272 missing values, which are year-location with no issues, or no issues recorded, or no issues we searched for)
print(metadata['Issues'].value_counts(dropna=False))

#custom selected issues. no editing recquired
#order of issue could affect target + so no extra editing or grouping

Issues
NaN                                                       178
equipment                                                  21
water                                                       8
weed                                                        7
lodg                                                        3
equipment, water                                            3
animal                                                      3
stand                                                       3
water, lodg                                                 2
germination                                                 2
animal, equipment                                           2
lodg, equipment                                             2
equipment, weed                                             2
equipment, equipment                                        2
stand, water                                                2
germination, stand, equipment, equipment                    2
l

In [41]:
#Type of Planter (25/272 missing values)
print(metadata['Type_of_planter (fluted cone; belt cone; air planter)'].value_counts(dropna=False))

Type_of_planter (fluted cone; belt cone; air planter)
air planter                                       151
fluted cone                                        36
belt cone                                          26
NaN                                                25
Air planter                                         7
SRES Air                                            3
vacuum precision planter                            3
air                                                 2
Almaco TP2                                          2
Fluted cone                                         2
4 row almaco GPS Drop precision vacuum planter      2
Almaco Seed Pro 360                                 2
Almaco 4-row air planter                            2
Fluted Cone                                         1
air (Seedpro)                                       1
John Deere 7300                                     1
vacuum planter                                      1
Vacuum                      

In [42]:
#convert all strings to lower case
metadata['Type_of_planter (fluted cone; belt cone; air planter)'] = metadata['Type_of_planter (fluted cone; belt cone; air planter)'].str.lower()

In [43]:
#fix vacum typo to vacuum
metadata['Type_of_planter (fluted cone; belt cone; air planter)'] = metadata['Type_of_planter (fluted cone; belt cone; air planter)'].replace('vacum', 'vacuum')
#fix flute cone typo to fluted cone
metadata['Type_of_planter (fluted cone; belt cone; air planter)'] = metadata['Type_of_planter (fluted cone; belt cone; air planter)'].replace('flute cone', 'fluted cone')
#change air string to air planter
metadata['Type_of_planter (fluted cone; belt cone; air planter)'] = metadata['Type_of_planter (fluted cone; belt cone; air planter)'].replace('air', 'air planter')

In [44]:
#col: irrigated (222 missing values, could be no irrigation and/or not recorded)
print(metadata['Irrigated'].value_counts(dropna=False))
#are NaN unirrigated. Most likely but should I make assumption
#for now I am not**

Irrigated
NaN    222
no      31
yes     19
Name: count, dtype: int64


In [45]:
#convert all NaN in metadata df to n/a (not applicable)
#assume missing values are missing because feature was n/a
#could also be not recorded (data collection and recording problem)
metadata = metadata.fillna('n/a')

In [46]:
metadata.head(5)

,Year,Env,Experiment_Code,Treatment,"Soil_Taxonomic_ID and horizon description, if known",Previous_Crop,Pre-plant_tillage_method(s),In-season_tillage_method(s),Issues,lon,lat,Type_of_planter (fluted cone; belt cone; air planter),Irrigated
0,2014,DEH1_2014,DEH1,n/a,n/a,Soybeans,conventional,n/a,n/a,n/a,n/a,air planter,n/a
1,2014,GAH1_2014,GAH1,n/a,n/a,cotton,conventional,n/a,n/a,n/a,n/a,fluted cone,n/a
2,2014,IAH1a_2014,IAH1,n/a,n/a,Soybeans,field cultivator,n/a,n/a,n/a,n/a,air planter,n/a
3,2014,IAH1b_2014,IAH1,n/a,n/a,Soybeans,field cultivator,n/a,n/a,n/a,n/a,air planter,n/a
4,2014,IAH1c_2014,IAH1,n/a,n/a,Soybeans,field cultivator,n/a,n/a,n/a,n/a,air planter,n/a


In [47]:
metadata.to_csv('extracted_metadata_features_with_missing_values_filled_in.csv', index=False)

In [6]:
missing_values_2 = metadata.isnull().sum()
print(f"Missing values per column:\n{missing_values_2}")
#no missing values

Missing values per column:
Year                                                     0
Env                                                      0
Experiment_Code                                          0
Treatment                                                0
Soil_Taxonomic_ID and horizon description, if known      0
Previous_Crop                                            0
Pre-plant_tillage_method(s)                              0
In-season_tillage_method(s)                              0
Issues                                                   0
lon                                                      0
lat                                                      0
Type_of_planter (fluted cone; belt cone; air planter)    0
Irrigated                                                0
dtype: int64


#### Find substitute year-locations to fill in missing lat and lon values
* Year-locations grown in same city was used as subsitutes for year-locations with missing lat and lon measurements

For year-locations with no substitute found:
* Either lat/lons were imputed manually after searching (if city was known) or
* An year-location in same region was used as substitute (less confidence in subs)

In [46]:
metadata = pd.read_csv('extracted_metadata_features_with_missing_values_filled_in.csv',keep_default_na=False)
#keep n/a as a string

In [47]:
metadata.head(5)

,Year,Env,Experiment_Code,Treatment,"Soil_Taxonomic_ID and horizon description, if known",Previous_Crop,Pre-plant_tillage_method(s),In-season_tillage_method(s),Issues,lon,lat,Type_of_planter (fluted cone; belt cone; air planter),Irrigated
0,2014,DEH1_2014,DEH1,n/a,n/a,Soybeans,conventional,n/a,n/a,n/a,n/a,air planter,n/a
1,2014,GAH1_2014,GAH1,n/a,n/a,cotton,conventional,n/a,n/a,n/a,n/a,fluted cone,n/a
2,2014,IAH1a_2014,IAH1,n/a,n/a,Soybeans,field cultivator,n/a,n/a,n/a,n/a,air planter,n/a
3,2014,IAH1b_2014,IAH1,n/a,n/a,Soybeans,field cultivator,n/a,n/a,n/a,n/a,air planter,n/a
4,2014,IAH1c_2014,IAH1,n/a,n/a,Soybeans,field cultivator,n/a,n/a,n/a,n/a,air planter,n/a


In [48]:
#extract rows with missing lat and lon values
missing_lat_lon = metadata[(metadata['lat'] == 'n/a') & (metadata['lon'] == 'n/a')]
print(len(missing_lat_lon))

#list of envs with missing lat and lon measurements
missing_env_list = missing_lat_lon['Env'].tolist()

#all 52 missing envs selected

52


In [49]:
missing_env_list
#missing lat and lon measurements across many years. mainly 2014.

['DEH1_2014',
 'GAH1_2014',
 'IAH1a_2014',
 'IAH1b_2014',
 'IAH1c_2014',
 'IAH2_2014',
 'IAH3_2014',
 'IAH4_2014',
 'ILH1_2014',
 'INH1_2014',
 'MNH1_2014',
 'MOH1_2014',
 'MOH2_2014',
 'NCH1_2014',
 'NEH1_2014',
 'NEH2_2014',
 'NEH3_2014',
 'NYH1_2014',
 'NYH2_2014',
 'ONH1_2014',
 'ONH2_2014',
 'TXH1_2014',
 'TXH2_2014',
 'WIH1_2014',
 'NYH1_2015',
 'TXH2_2015',
 'IAH1_2016',
 'IAH2_2016',
 'IAH3_2016',
 'IAH4_2016',
 'NEH1_2016',
 'NEH4_2016',
 'NYH1_2016',
 'TXH2_2016',
 'ILH1_2017',
 'INH1_2017',
 'MNH1_2017',
 'TXH1-Dry_2017',
 'TXH1-Early_2017',
 'TXH1-Late_2017',
 'TXH2_2017',
 'IAH1_2018',
 'IAH2_2018',
 'IAH3_2018',
 'IAH4_2018',
 'TXH2_2018',
 'NEH2_2019',
 'TXH4_2019',
 'NEH2_2020',
 'NEH3_2020',
 'ILH1_2021',
 'NYS1_2021']

In [50]:
#import original metadata file with City info
metadata_org = pd.read_csv('2_Training_Meta_Data_2014_2023.csv')

In [51]:
metadata_org.head(5)

,Year,Env,Experiment_Code,Treatment,City,Farm,Field,Trial_ID (Assigned by collaborator for internal reference),"Soil_Taxonomic_ID and horizon description, if known","Weather_Station_Serial_Number (Last four digits, e.g. m2700s#####)",...,Cardinal_Heading_Pass_1,Irrigated,Issue/comment_#1,Issue/comment_#2,Issue/comment_#3,Issue/comment_#4,Issue/comment_#5,Issue/comment_#6,Issue/comment_#7,Issue/comment_#8
0,2014,DEH1_2014,DEH1,NaN,Georgetown,Elbert N. & Ann V. Carvel Research & Education...,27AB,NaN,NaN,9079,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014,GAH1_2014,GAH1,NaN,Tifton,USDA - Bellflower experimental farm,18,NaN,NaN,8427,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014,IAH1a_2014,IAH1,NaN,Ames,Worle,NaN,NaN,NaN,9080,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Information for IAH1a_2014, IAH1b_2014,and IAH...",NaN
3,2014,IAH1b_2014,IAH1,NaN,Ames,Worle,NaN,NaN,NaN,9080,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Information for IAH1a_2014, IAH1b_2014,and IAH...",NaN
4,2014,IAH1c_2014,IAH1,NaN,Ames,Worle,NaN,NaN,NaN,9080,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Information for IAH1a_2014, IAH1b_2014,and IAH...",NaN


In [52]:
#create a dataframe with Env, City columns of instances WITHOUT lat and lon measurements
missing_env_cities = metadata_org[metadata_org['Env'].isin(missing_env_list)][['Env', 'City']]

In [53]:
missing_env_cities.head(5)

,Env,City
0,DEH1_2014,Georgetown
1,GAH1_2014,Tifton
2,IAH1a_2014,Ames
3,IAH1b_2014,Ames
4,IAH1c_2014,Ames


In [54]:
#create a dataframe with Env, City columns of instances WITH lat and lon measurement
#This serves as a pool of potential substitute envs
sub_pool_env_cities = metadata_org[
    metadata_org['Latitude_of_Field_Corner_#1 (lower left)'].notna() & metadata_org['Longitude_of_Field_Corner_#1 (lower left)'].notna()][['Env', 'City']]

In [55]:
len(sub_pool_env_cities), len(missing_env_cities)
#272 year-locations alltogether (see above)
#all envs other than 52 missing envs are in potential substitute pool

(220, 52)

In [56]:
#find substitute envs from sub pool using pandas merge function
substitute_envs = missing_env_cities.merge(
    sub_pool_env_cities,
    on='City',
    how='left',  #ensures envs without substitutes are retained
    suffixes=('_missing', '_substitute')
)

In [57]:
substitute_envs.head(5)
#mulitple subs from across different years found 

,Env_missing,City,Env_substitute
0,DEH1_2014,Georgetown,DEH1_2015
1,DEH1_2014,Georgetown,DEH1_2016
2,DEH1_2014,Georgetown,DEH1_2017
3,DEH1_2014,Georgetown,DEH1_2018
4,DEH1_2014,Georgetown,DEH1_2019


In [58]:
#select only first substitute env for those with many substitutes.
first_subs_envs = substitute_envs_by_city.groupby('Env_missing').first().reset_index()

In [59]:
len(first_subs_envs)
#all 52 missing year-locations in df

52

In [97]:
first_subs_envs.to_csv('subs_lat_lon_metadata.csv', index=False)

In [60]:
first_subs_envs.head(5)

,Env_missing,City,Env_substitute
0,DEH1_2014,Georgetown,DEH1_2015
1,GAH1_2014,Tifton,GAH1_2015
2,IAH1_2016,Crawfordsville,IAH4_2015
3,IAH1_2018,Crawfordsville,IAH4_2015
4,IAH1a_2014,Ames,IAH1_2015


In [61]:
#check if any missing year-locations have no subs
print('missing cities/subs:')
first_subs_envs.isna().sum()
#5 missing cities
#8 missing env with no subs

missing cities/subs:


Env_missing       0
City              5
Env_substitute    8
dtype: int64

In [62]:
#find year-locations with no subs found or no city info
still_missing = first_subs_envs[
    first_subs_envs['City'].isna() | first_subs_envs['Env_substitute'].isna()]

In [63]:
still_missing
#Texas why you always a problem for me
#8 instances no subs:
#In 5 instances, no cities given 
#In 3 instances, cities are given. Lubbock and Halfway are cities in Texas. find these lats/lons by searching.
#what do do with rest 5?

,Env_missing,City,Env_substitute
17,ILH1_2017,None,None
20,INH1_2017,None,None
45,TXH2_2014,Halfway,None
46,TXH2_2015,None,None
47,TXH2_2016,None,None
48,TXH2_2017,None,None
49,TXH2_2018,Lubbock,None
50,TXH4_2019,Lubbock,None


In [71]:
#find more info on ILH1
metadata_org[metadata_org['Experiment_Code'] == 'ILH1']
#looks like after 2016 ILHI has been in Champaign
#therefore let ILH1_2017 substitute env be ILH1_2018 

,Year,Env,Experiment_Code,Treatment,City,Farm,Field,Trial_ID (Assigned by collaborator for internal reference),"Soil_Taxonomic_ID and horizon description, if known","Weather_Station_Serial_Number (Last four digits, e.g. m2700s#####)",...,Cardinal_Heading_Pass_1,Irrigated,Issue/comment_#1,Issue/comment_#2,Issue/comment_#3,Issue/comment_#4,Issue/comment_#5,Issue/comment_#6,Issue/comment_#7,Issue/comment_#8
8,2014,ILH1_2014,ILH1,NaN,Urbana,Maxwell Farms,MF500,NaN,NaN,8653,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,2015,ILH1_2015,ILH1,NaN,Urbana,NaN,NaN,NaN,NaN,8653,...,180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,2016,ILH1_2016,ILH1,Standard,Champaign,Maxwell Farm,MF-500,NaN,Flanagan Silt Loam (154A),8653,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,2017,ILH1_2017,ILH1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117,2018,ILH1_2018,ILH1,Standard,Champaign,Maxwell Farm,MF200,ILH1,Flanagan Silt Loam,8653,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
145,2019,ILH1_2019,ILH1,Standard,Champaign,Maxwell Farm,MF1000,ILH1,Flanagan Silt Loam,8653,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199,2021,ILH1_2021,ILH1,Standard,Champaign,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
226,2022,ILH1_2022,ILH1,Standard,Champaign,South Farms,AnSci200,NaN,NaN,8653,...,NaN,no,Link to additional weather source available on...,"In general, dry and hot summer.",NaN,NaN,NaN,NaN,NaN,NaN
253,2023,ILH1_2023,ILH1,Standard,Champaign,South Farms,S700,NaN,"Drummer silty clay loam, 0 to 2 percent slopes...",8653,...,NaN,no,Link to additional weather source available on...,"In general, dry and hot summer.",NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
#more info on INH1
metadata_org[metadata_org['Experiment_Code'] == 'INH1']
#looks like INH1 is always in West Lafayette and farm is ACRE
#therefore let INH1_2017 sub be INH1_2018

,Year,Env,Experiment_Code,Treatment,City,Farm,Field,Trial_ID (Assigned by collaborator for internal reference),"Soil_Taxonomic_ID and horizon description, if known","Weather_Station_Serial_Number (Last four digits, e.g. m2700s#####)",...,Cardinal_Heading_Pass_1,Irrigated,Issue/comment_#1,Issue/comment_#2,Issue/comment_#3,Issue/comment_#4,Issue/comment_#5,Issue/comment_#6,Issue/comment_#7,Issue/comment_#8
9,2014,INH1_2014,INH1,NaN,West Lafayette,Purdue ACRE,97/98,NaN,NaN,8657,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,2015,INH1_2015,INH1,NaN,West Lafayette,NaN,NaN,NaN,NaN,8657,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,2016,INH1_2016,INH1,NaN,West Lafayette,Purdue ACRE,54 North,NaN,NaN,8657,...,0,NaN,"At planting, a row unit plugged up with a clod...",July 14. Noticed that Moisture sensor wire was...,NaN,NaN,NaN,NaN,NaN,NaN
89,2017,INH1_2017,INH1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118,2018,INH1_2018,INH1,Standard,West Lafayette,Purdue ACRE,54NN,GxE,RcA-Raub-Brenton Complex,8657,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
146,2019,INH1_2019,INH1,Standard,West Lafayette,Purdue ACRE,42,INH1,Raub-Brenton complex,8657,...,0,NaN,Bulk Density sensor started actigng up during ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170,2020,INH1_2020,INH1,Standard,West Lafayette,Purdue ACRE,54N,G2F,Raub-Brenton complex,8657,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200,2021,INH1_2021,INH1,Standard,West Lafayette,Purdue ACRE,42,GxE,Chalmers Silty Clay Loam,8657,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227,2022,INH1_2022,INH1,Standard,West Lafayette,ACRE,78,GxE,Raub/Brenton Complex,8657,...,0,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
254,2023,INH1_2023,INH1,Standard,West Lafayette,ACRE,42,GxE,Chalmers Silty Clay Loam,8657,...,0,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
#more info on TXH2
metadata_org[metadata_org['Experiment_Code'] == 'TXH2']
#same problem as weather subs: could be Halfway, Lubbock or College Station (no consistency during 2015-2017 period)
#could either assume mode: College station or assume previous year: Halfway
#lets remain consistent with weather sub and assume College Station
#subs for TXH2_2015,2016,2017 is TXH2_2019

,Year,Env,Experiment_Code,Treatment,City,Farm,Field,Trial_ID (Assigned by collaborator for internal reference),"Soil_Taxonomic_ID and horizon description, if known","Weather_Station_Serial_Number (Last four digits, e.g. m2700s#####)",...,Cardinal_Heading_Pass_1,Irrigated,Issue/comment_#1,Issue/comment_#2,Issue/comment_#3,Issue/comment_#4,Issue/comment_#5,Issue/comment_#6,Issue/comment_#7,Issue/comment_#8
22,2014,TXH2_2014,TXH2,NaN,Halfway,Halfway,pivot,NaN,NaN,8630,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,2015,TXH2_2015,TXH2,NaN,NaN,NaN,NaN,NaN,NaN,8630,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,2016,TXH2_2016,TXH2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,2017,TXH2_2017,TXH2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134,2018,TXH2_2018,TXH2,Standard,Lubbock,NaN,NaN,NaN,NaN,8630,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
160,2019,TXH2_2019,TXH2,Standard,College Station,NaN,NaN,NaN,NaN,14790,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
185,2020,TXH2_2020,TXH2,Dry Land,College Station,NaN,NaN,NaN,NaN,14790,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212,2021,TXH2_2021,TXH2,Dryland optimal,College Station,Texas A&M Research Farm,302,G2F2,Belk Clay Loam (BaA) / Ships Clay Loam (ShA),14790,...,310,NaN,There was a substantial morning glory infestation,Coordinates approximate from Google Maps,NaN,NaN,NaN,NaN,NaN,NaN
240,2022,TXH2_2022,TXH2,Drought,College Station,Texas A&M Research Farm,302,TXH,NaN,14790,...,312,no,The post harvest herbicide spray was to get ah...,Plots 325-500 where irrigated due to a busted ...,NaN,NaN,NaN,NaN,NaN,NaN
267,2023,TXH2_2023,TXH2,Drought,College Station,Texas Research Farm,224,TXH2,NaN,NaN,...,45,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
#Add missing envs and subs into first_subs_envs df

# Define cities and env subs for 5 missing year-locations in a dictionary
subs_round_2 = {
    'ILH1_2017': {'City': 'Champaign', 'Env_substitute': 'ILH1_2018'},
    'INH1_2017': {'City': 'West Lafayette', 'Env_substitute': 'INH1_2018'},
    'TXH2_2015': {'City': 'College Station', 'Env_substitute': 'TXH2_2019'},
    'TXH2_2016': {'City': 'College Station', 'Env_substitute': 'TXH2_2019'},
    'TXH2_2017': {'City': 'College Station', 'Env_substitute': 'TXH2_2019'},
}

#loops through each specified env and searches for them in first_subs_envs df
for env, info in subs_round_2.items():
    first_subs_envs.loc[first_subs_envs['Env_missing'] == env, 'City'] = info['City'] #add respective city info if env matches env in dictionary
    first_subs_envs.loc[first_subs_envs['Env_missing'] == env, 'Env_substitute'] = info['Env_substitute'] #add sub info in same way


In [77]:
first_subs_envs.isna().sum()
#3 remaining missing year-locations, are ones where city is known.

Env_missing       0
City              0
Env_substitute    3
dtype: int64

In [80]:
first_subs_envs_v2 = first_subs_envs.dropna()
#remember TXH2_2014,2018,TXH4_2019 need to be added after internet search. remove them for now

len(first_subs_envs_v2)
#49 subs found for lat/lon imputation. 

49

In [81]:
first_subs_envs_v2.to_csv('most_subs_lat_lon_metadata.csv', index=False)

In [86]:
#search and find coordinates for TXH2_2018, TXH4_2019, TXH2_2014
still_missing

,Env_missing,City,Env_substitute
17,ILH1_2017,None,None
20,INH1_2017,None,None
45,TXH2_2014,Halfway,None
46,TXH2_2015,None,None
47,TXH2_2016,None,None
48,TXH2_2017,None,None
49,TXH2_2018,Lubbock,None
50,TXH4_2019,Lubbock,None


In [83]:
#determien how lat and lon are reported
metadata.tail(5)

#lat  -90 to 90 (south-north) 
#long -180 to 180 (west-east) 

#Lubbock coordinates: lat= 33.5779, lon= -101.8552 for TXH2_2018, TXH4_2019 
#ref: https://www.latlong.net/place/lubbock-tx-usa-1172.html#:~:text=Lubbock%2C%20TX%2C%20USA%20Lat%20Long,°%2051%27%2018.2592%27%27%20W.

#Halfway coordinates: lat=34.1881, lon=-101.9524 for TXH2_201411
#ref: https://g.co/kgs/XvDuHWH

,Year,Env,Experiment_Code,Treatment,"Soil_Taxonomic_ID and horizon description, if known",Previous_Crop,Pre-plant_tillage_method(s),In-season_tillage_method(s),Issues,lon,lat,Type_of_planter (fluted cone; belt cone; air planter),Irrigated
267,2023,TXH2_2023,TXH2,Drought,n/a,Sorghum,discing,cultivator,n/a,-96.43165871,30.55034681,belt cone,no
268,2023,TXH3_2023,TXH3,Late Planting,n/a,Sorghum,disicing,cultivator,n/a,-96.4310292925,30.549647405,belt cone,yes
269,2023,WIH1_2023,WIH1,Standard,"PoA, TrB",Soybeans,field cultivator,n/a,n/a,-89.5297795,43.0554021125,air planter,no
270,2023,WIH2_2023,WIH2,Standard,"PnA, PnB",Soybeans,field cultivator,n/a,equipment,-89.3841457,43.304242975,air planter,no
271,2023,WIH3_2023,WIH3,Standard,Sandy,n/a,"disc, dynadrive",n/a,equipment,-89.54400899999999,44.115644625,air planter,yes


In [108]:
# Create custom dict with lat and lon info for 3 missing year-locations above
lat_lon_info = {
    'TXH2_2018': {'Lat': 33.5779, 'Lon': -101.8552},  # Lubbock
    'TXH4_2019': {'Lat': 33.5779, 'Lon': -101.8552},  # Lubbock
    'TXH2_2014': {'Lat': 34.1881, 'Lon': -101.9524},  # Halfway
}

# Impute in missing lat and lon for above year-locations only directly into metadata df
for env, coords in lat_lon_info.items():
    metadata.loc[metadata['Env'] == env, 'lat'] = coords['Lat']
    metadata.loc[metadata['Env'] == env, 'lon'] = coords['Lon']

In [90]:
#impute lat and lon for rest of missing year-locations using subs
first_subs_envs_v2.head(5)

,Env_missing,City,Env_substitute
0,DEH1_2014,Georgetown,DEH1_2015
1,GAH1_2014,Tifton,GAH1_2015
2,IAH1_2016,Crawfordsville,IAH4_2015
3,IAH1_2018,Crawfordsville,IAH4_2015
4,IAH1a_2014,Ames,IAH1_2015


In [109]:
# Iterate over each row in first_subs_envs_v2
for _, row in first_subs_envs_v2.iterrows():
    env_missing = row['Env_missing'] #define missing env
    env_substitute = row['Env_substitute'] #define sub for missing env
    
    # Get lat and lon of the substitute env from metadata
    lat_substitute = metadata.loc[metadata['Env'] == env_substitute, 'lat'].values #find and store subs lat value
    lon_substitute = metadata.loc[metadata['Env'] == env_substitute, 'lon'].values #find and store subs lon value
    
    # Impute lat and lon for the missing env in metadata
    metadata.loc[metadata['Env'] == env_missing, 'lat'] = lat_substitute #replace missing lat with sub
    metadata.loc[metadata['Env'] == env_missing, 'lon'] = lon_substitute #replace missing lon with sub

In [110]:
metadata.head(5)
#first 5 have been imputed

,Year,Env,Experiment_Code,Treatment,"Soil_Taxonomic_ID and horizon description, if known",Previous_Crop,Pre-plant_tillage_method(s),In-season_tillage_method(s),Issues,lon,lat,Type_of_planter (fluted cone; belt cone; air planter),Irrigated
0,2014,DEH1_2014,DEH1,n/a,n/a,Soybeans,conventional,n/a,n/a,-75.4656925,38.62935675,air planter,n/a
1,2014,GAH1_2014,GAH1,n/a,n/a,cotton,conventional,n/a,n/a,-83.555095,31.505771,fluted cone,n/a
2,2014,IAH1a_2014,IAH1,n/a,n/a,Soybeans,field cultivator,n/a,n/a,-93.693369725,41.99809519750001,air planter,n/a
3,2014,IAH1b_2014,IAH1,n/a,n/a,Soybeans,field cultivator,n/a,n/a,-93.693369725,41.99809519750001,air planter,n/a
4,2014,IAH1c_2014,IAH1,n/a,n/a,Soybeans,field cultivator,n/a,n/a,-93.693369725,41.99809519750001,air planter,n/a


In [111]:
metadata['lat'].isna().sum(), metadata['lon'].isna().sum()
#no NaN created

(np.int64(0), np.int64(0))

In [112]:
metadata['lat'].str.contains('n/a').sum(),metadata['lon'].str.contains('n/a').sum()
#all lats and lons imputed

(0, 0)

In [113]:
metadata.to_csv('extracted_metadata_features_lat_lon_imputed_missing_values_filled_in.csv',index=False)

In [114]:
len(metadata)
#all 272 year-locations preserved

272

#### Label Encode and/or Normalise all metadata/management features

In [2]:
metadata = pd.read_csv('extracted_metadata_features_lat_lon_imputed_missing_values_filled_in.csv', keep_default_na=False)

In [3]:
metadata.columns

Index(['Year', 'Env', 'Experiment_Code', 'Treatment',
       'Soil_Taxonomic_ID and horizon description, if known', 'Previous_Crop',
       'Pre-plant_tillage_method(s)', 'In-season_tillage_method(s)', 'Issues',
       'lon', 'lat', 'Type_of_planter (fluted cone; belt cone; air planter)',
       'Irrigated'],
      dtype='object')

In [4]:
metadata.head(5)

,Year,Env,Experiment_Code,Treatment,"Soil_Taxonomic_ID and horizon description, if known",Previous_Crop,Pre-plant_tillage_method(s),In-season_tillage_method(s),Issues,lon,lat,Type_of_planter (fluted cone; belt cone; air planter),Irrigated
0,2014,DEH1_2014,DEH1,n/a,n/a,Soybeans,conventional,n/a,n/a,-75.465693,38.629357,air planter,n/a
1,2014,GAH1_2014,GAH1,n/a,n/a,cotton,conventional,n/a,n/a,-83.555095,31.505771,fluted cone,n/a
2,2014,IAH1a_2014,IAH1,n/a,n/a,Soybeans,field cultivator,n/a,n/a,-93.693370,41.998095,air planter,n/a
3,2014,IAH1b_2014,IAH1,n/a,n/a,Soybeans,field cultivator,n/a,n/a,-93.693370,41.998095,air planter,n/a
4,2014,IAH1c_2014,IAH1,n/a,n/a,Soybeans,field cultivator,n/a,n/a,-93.693370,41.998095,air planter,n/a


In [6]:
metadata.dtypes
#only label encode metdata cols

Year                                                       int64
Env                                                       object
Experiment_Code                                           object
Treatment                                                 object
Soil_Taxonomic_ID and horizon description, if known       object
Previous_Crop                                             object
Pre-plant_tillage_method(s)                               object
In-season_tillage_method(s)                               object
Issues                                                    object
lon                                                      float64
lat                                                      float64
Type_of_planter (fluted cone; belt cone; air planter)     object
Irrigated                                                 object
dtype: object

In [42]:
from sklearn.preprocessing import LabelEncoder

#create a function to label encode cols in any df
def label_encode_features(df: pd.DataFrame, exclude_cols: list = None):
    """
    Label encodes all object columns in a dataframe, excluding any specified in exclude_cols.
    Deletes the original columns after encoding.
        
    Returns:
        pd.DataFrame: DataFrame with label-encoded columns replacing orginal columns
    """
    df = df.copy()
    le = LabelEncoder()
    
    if exclude_cols is None:
        exclude_cols = []
    
    for col in df.columns:
        if col not in exclude_cols and df[col].dtype == 'object': #only proceed if cols are objects
            df[col + '_Label'] = le.fit_transform(df[col].astype(str)) #converts values in object cols to strings
            df.drop(columns=col, inplace=True)
    
    return df


In [43]:
#apply function
metadata_encoded = label_encode_features(metadata, exclude_cols=['Env','Year','lat', 'lon']) 
#do not label encode numeric features + Env for merging

In [44]:
metadata_encoded.head(5)

,Year,Env,lon,lat,Experiment_Code_Label,Treatment_Label,"Soil_Taxonomic_ID and horizon description, if known_Label",Previous_Crop_Label,Pre-plant_tillage_method(s)_Label,In-season_tillage_method(s)_Label,Issues_Label,Type_of_planter (fluted cone; belt cone; air planter)_Label,Irrigated_Label
0,2014,DEH1_2014,-75.465693,38.629357,3,11,48,6,23,18,24,2,0
1,2014,GAH1_2014,-83.555095,31.505771,4,11,48,10,23,18,24,7,0
2,2014,IAH1a_2014,-93.693370,41.998095,7,11,48,6,69,18,24,2,0
3,2014,IAH1b_2014,-93.693370,41.998095,7,11,48,6,69,18,24,2,0
4,2014,IAH1c_2014,-93.693370,41.998095,7,11,48,6,69,18,24,2,0


In [45]:
print('file saving ...')
metadata_encoded.to_csv('metadata_label_encoded.csv', index=False)
print('file saved')

file saving ...
file saved


In [46]:
from sklearn.preprocessing import MinMaxScaler

def normalize_features(df: pd.DataFrame, exclude_cols: list = None):
    """
    Normalizes all numeric columns in a dataframe using MinMaxScaler, excluding any specified in exclude_cols. 
    Deletes the original columns after normalization.
    
    Returns:
        pd.DataFrame: DataFrame with normalized columns replacing original columns
    """
    df = df.copy()
    scaler = MinMaxScaler()
    
    if exclude_cols is None:
        exclude_cols = []

    for col in df.select_dtypes(include=['number']).columns: #only proceed if features are numeric regardless of whether it is int or float
        if col not in exclude_cols:
            df[col + '_Normalized'] = scaler.fit_transform(df[[col]])
            df.drop(columns=col, inplace=True)

    return df


In [47]:
metadata_encoded_normalised = normalize_features(metadata_encoded, exclude_cols=['Env']) #do not normalise for merging

In [48]:
metadata_encoded_normalised.head(5)

,Env,Year_Normalized,lon_Normalized,lat_Normalized,Experiment_Code_Label_Normalized,Treatment_Label_Normalized,"Soil_Taxonomic_ID and horizon description, if known_Label_Normalized",Previous_Crop_Label_Normalized,Pre-plant_tillage_method(s)_Label_Normalized,In-season_tillage_method(s)_Label_Normalized,Issues_Label_Normalized,Type_of_planter (fluted cone; belt cone; air planter)_Label_Normalized,Irrigated_Label_Normalized
0,DEH1_2014,0.0,0.256959,0.385771,0.069767,1.0,0.979592,0.352941,0.247312,0.857143,0.533333,0.133333,0.0
1,GAH1_2014,0.0,0.186578,0.046148,0.093023,1.0,0.979592,0.588235,0.247312,0.857143,0.533333,0.466667,0.0
2,IAH1a_2014,0.0,0.098371,0.546379,0.162791,1.0,0.979592,0.352941,0.741935,0.857143,0.533333,0.133333,0.0
3,IAH1b_2014,0.0,0.098371,0.546379,0.162791,1.0,0.979592,0.352941,0.741935,0.857143,0.533333,0.133333,0.0
4,IAH1c_2014,0.0,0.098371,0.546379,0.162791,1.0,0.979592,0.352941,0.741935,0.857143,0.533333,0.133333,0.0


In [49]:
metadata_encoded_normalised.to_csv('metadata_label_encoded_normalised.csv', index=False)

In [2]:
#merge with weather and snp data

#import snp_weather label encoded/normalised csv
print('importing files...')
snp_weather = pd.read_csv('final_snp_weather_trait_data_label_encoded_year_and_labels_normalised.csv')
metadata_encoded_normalised = pd.read_csv('metadata_label_encoded_normalised.csv')
print('files imported')

importing files...
files imported


In [3]:
snp_weather.head(5)
#Unnamed:0 because did not do index=False when saving as csv

,Unnamed: 0,Env,Year,Hybrid,Yield_Mg_ha,S1_1007742,S1_1020677,S1_2018002,S1_2101934,S1_2275970,...,T2M_GDD - Week 31,T2M_MAX - Week 31,T2M_MIN - Week 31,WS2M - Week 31,Growth_Cycle_Weeks,Hybrid_Label,Env_Label,Year_Normalized,Hybrid_Label_Normalized,Env_Label_Normalized
0,0,DEH1_2014,2014,M0088/LH185,5.721725,0.5,0.5,0.0,0.5,0.5,...,0.0,0.0,0.0,0.0,21.0,1123,10,0.0,0.227466,0.037037
1,1,DEH1_2014,2014,M0143/LH185,11.338246,0.0,0.5,0.0,0.5,0.5,...,0.0,0.0,0.0,0.0,21.0,1149,10,0.0,0.232732,0.037037
2,2,DEH1_2014,2014,M0003/LH185,6.540810,0.5,0.5,0.0,0.5,0.5,...,0.0,0.0,0.0,0.0,21.0,992,10,0.0,0.200932,0.037037
3,3,DEH1_2014,2014,M0035/LH185,10.366857,0.5,0.5,0.0,0.5,0.5,...,0.0,0.0,0.0,0.0,21.0,1062,10,0.0,0.215110,0.037037
4,4,DEH1_2014,2014,M0052/LH185,10.908814,0.5,0.5,0.0,0.5,0.5,...,0.0,0.0,0.0,0.0,21.0,1084,10,0.0,0.219567,0.037037


In [3]:
#drop Hybrid, Year,Env_Label, Hybrid_Label
#remove Env after merging
snp_weather = snp_weather.drop(columns=['Year', 'Hybrid', 'Env_Label', 'Hybrid_Label'])

In [4]:
#drop unnamed:0
snp_weather = snp_weather.drop(columns=['Unnamed: 0'])

In [5]:
len(snp_weather)
#162535 instances

162535

In [6]:
snp_weather.head(5)

,Env,Yield_Mg_ha,S1_1007742,S1_1020677,S1_2018002,S1_2101934,S1_2275970,S1_2800964,S1_2811950,S1_2888631,...,T2MDEW - Week 31,T2M_DIFF_SUM - Week 31,T2M_GDD - Week 31,T2M_MAX - Week 31,T2M_MIN - Week 31,WS2M - Week 31,Growth_Cycle_Weeks,Year_Normalized,Hybrid_Label_Normalized,Env_Label_Normalized
0,DEH1_2014,5.721725,0.5,0.5,0.0,0.5,0.5,0.0,0.0,0.5,...,0.0,0.0,0.0,0.0,0.0,0.0,21.0,0.0,0.227466,0.037037
1,DEH1_2014,11.338246,0.0,0.5,0.0,0.5,0.5,0.5,0.5,0.5,...,0.0,0.0,0.0,0.0,0.0,0.0,21.0,0.0,0.232732,0.037037
2,DEH1_2014,6.540810,0.5,0.5,0.0,0.5,0.5,0.0,0.0,0.5,...,0.0,0.0,0.0,0.0,0.0,0.0,21.0,0.0,0.200932,0.037037
3,DEH1_2014,10.366857,0.5,0.5,0.0,0.5,0.5,0.0,0.0,0.5,...,0.0,0.0,0.0,0.0,0.0,0.0,21.0,0.0,0.215110,0.037037
4,DEH1_2014,10.908814,0.5,0.5,0.0,0.5,0.5,0.0,0.0,0.5,...,0.0,0.0,0.0,0.0,0.0,0.0,21.0,0.0,0.219567,0.037037


In [8]:
metadata_encoded_normalised.head(5)

,Env,Year_Normalized,lon_Normalized,lat_Normalized,Experiment_Code_Label_Normalized,Treatment_Label_Normalized,"Soil_Taxonomic_ID and horizon description, if known_Label_Normalized",Previous_Crop_Label_Normalized,Pre-plant_tillage_method(s)_Label_Normalized,In-season_tillage_method(s)_Label_Normalized,Issues_Label_Normalized,Type_of_planter (fluted cone; belt cone; air planter)_Label_Normalized,Irrigated_Label_Normalized
0,DEH1_2014,0.0,0.256959,0.385771,0.069767,1.0,0.979592,0.352941,0.247312,0.857143,0.533333,0.133333,0.0
1,GAH1_2014,0.0,0.186578,0.046148,0.093023,1.0,0.979592,0.588235,0.247312,0.857143,0.533333,0.466667,0.0
2,IAH1a_2014,0.0,0.098371,0.546379,0.162791,1.0,0.979592,0.352941,0.741935,0.857143,0.533333,0.133333,0.0
3,IAH1b_2014,0.0,0.098371,0.546379,0.162791,1.0,0.979592,0.352941,0.741935,0.857143,0.533333,0.133333,0.0
4,IAH1c_2014,0.0,0.098371,0.546379,0.162791,1.0,0.979592,0.352941,0.741935,0.857143,0.533333,0.133333,0.0


In [7]:
#drop Year_Normalised because it is already in snp_weather
metadata_encoded_normalised = metadata_encoded_normalised.drop(columns=['Year_Normalized'])

In [8]:
metadata_encoded_normalised.columns
#merge on Env, add 11 management features

Index(['Env', 'lon_Normalized', 'lat_Normalized',
       'Experiment_Code_Label_Normalized', 'Treatment_Label_Normalized',
       'Soil_Taxonomic_ID and horizon description, if known_Label_Normalized',
       'Previous_Crop_Label_Normalized',
       'Pre-plant_tillage_method(s)_Label_Normalized',
       'In-season_tillage_method(s)_Label_Normalized',
       'Issues_Label_Normalized',
       'Type_of_planter (fluted cone; belt cone; air planter)_Label_Normalized',
       'Irrigated_Label_Normalized'],
      dtype='object')

In [9]:
snp_weather_management = pd.merge(snp_weather, metadata_encoded_normalised, on='Env', how='inner')

In [12]:
snp_weather_management.head(5)

,Env,Yield_Mg_ha,S1_1007742,S1_1020677,S1_2018002,S1_2101934,S1_2275970,S1_2800964,S1_2811950,S1_2888631,...,lat_Normalized,Experiment_Code_Label_Normalized,Treatment_Label_Normalized,"Soil_Taxonomic_ID and horizon description, if known_Label_Normalized",Previous_Crop_Label_Normalized,Pre-plant_tillage_method(s)_Label_Normalized,In-season_tillage_method(s)_Label_Normalized,Issues_Label_Normalized,Type_of_planter (fluted cone; belt cone; air planter)_Label_Normalized,Irrigated_Label_Normalized
0,DEH1_2014,5.721725,0.5,0.5,0.0,0.5,0.5,0.0,0.0,0.5,...,0.385771,0.069767,1.0,0.979592,0.352941,0.247312,0.857143,0.533333,0.133333,0.0
1,DEH1_2014,11.338246,0.0,0.5,0.0,0.5,0.5,0.5,0.5,0.5,...,0.385771,0.069767,1.0,0.979592,0.352941,0.247312,0.857143,0.533333,0.133333,0.0
2,DEH1_2014,6.540810,0.5,0.5,0.0,0.5,0.5,0.0,0.0,0.5,...,0.385771,0.069767,1.0,0.979592,0.352941,0.247312,0.857143,0.533333,0.133333,0.0
3,DEH1_2014,10.366857,0.5,0.5,0.0,0.5,0.5,0.0,0.0,0.5,...,0.385771,0.069767,1.0,0.979592,0.352941,0.247312,0.857143,0.533333,0.133333,0.0
4,DEH1_2014,10.908814,0.5,0.5,0.0,0.5,0.5,0.0,0.0,0.5,...,0.385771,0.069767,1.0,0.979592,0.352941,0.247312,0.857143,0.533333,0.133333,0.0


In [10]:
len(snp_weather_management)
#metadata info for all instances available

162535

In [11]:
from sklearn.preprocessing import MinMaxScaler
#normalise growth_cycle weeks which you forgot last time
scaler = MinMaxScaler()
snp_weather_management['Growth_Cycle_Weeks_normalized'] = scaler.fit_transform(snp_weather_management[['Growth_Cycle_Weeks']])

In [13]:
snp_weather_management = snp_weather_management.drop(columns=['Growth_Cycle_Weeks'])

In [15]:
print('saving large file')
snp_weather_management.to_csv('final_boss_snp_weather_management.csv', index=False)
print('large file saved')

saving large file
large file saved


#### Construct Random Forest Model to see improvement in model performance after addition of management features

In [2]:
print('importing large file...')
snp_weather_management = pd.read_csv('final_boss_snp_weather_management.csv')
print('large file imported')

importing large file...
large file imported


In [3]:
#seperate features and targets
X = snp_weather_management.drop('Yield_Mg_ha', axis=1)
y = snp_weather_management['Yield_Mg_ha']

In [6]:
len(X), len(y)
#162535

(162535, 162535)

In [17]:
#need to remove Env variable 
X = X.drop(columns=['Env'])

X.head(5)
#2546 features

,S1_1007742,S1_1020677,S1_2018002,S1_2101934,S1_2275970,S1_2800964,S1_2811950,S1_2888631,S1_3023078,S1_3027593,...,Experiment_Code_Label_Normalized,Treatment_Label_Normalized,"Soil_Taxonomic_ID and horizon description, if known_Label_Normalized",Previous_Crop_Label_Normalized,Pre-plant_tillage_method(s)_Label_Normalized,In-season_tillage_method(s)_Label_Normalized,Issues_Label_Normalized,Type_of_planter (fluted cone; belt cone; air planter)_Label_Normalized,Irrigated_Label_Normalized,Growth_Cycle_Weeks_normalized
0,0.5,0.5,0.0,0.5,0.5,0.0,0.0,0.5,0.0,0.5,...,0.069767,1.0,0.979592,0.352941,0.247312,0.857143,0.533333,0.133333,0.0,0.25
1,0.0,0.5,0.0,0.5,0.5,0.5,0.5,0.5,0.0,0.0,...,0.069767,1.0,0.979592,0.352941,0.247312,0.857143,0.533333,0.133333,0.0,0.25
2,0.5,0.5,0.0,0.5,0.5,0.0,0.0,0.5,0.0,0.5,...,0.069767,1.0,0.979592,0.352941,0.247312,0.857143,0.533333,0.133333,0.0,0.25
3,0.5,0.5,0.0,0.5,0.5,0.0,0.0,0.5,0.0,0.5,...,0.069767,1.0,0.979592,0.352941,0.247312,0.857143,0.533333,0.133333,0.0,0.25
4,0.5,0.5,0.0,0.5,0.5,0.0,0.0,0.5,0.0,0.5,...,0.069767,1.0,0.979592,0.352941,0.247312,0.857143,0.533333,0.133333,0.0,0.25


In [8]:
y.head(5)
#yield

0     5.721725
1    11.338246
2     6.540810
3    10.366857
4    10.908814
Name: Yield_Mg_ha, dtype: float64

In [18]:
#train-val-split
#import sklearn train_test_split
from sklearn.model_selection import train_test_split

#train-val 80-20 split
X_train, X_val, y_train, y_val = train_test_split(X,
                                                  y,
                                                  test_size=0.2,
                                                  random_state=42)

In [19]:
X_train.shape, y_train.shape
#130028 instances in training

((130028, 2546), (130028,))

In [20]:
X_val.shape, y_val.shape
#32507 instance in testing

((32507, 2546), (32507,))

In [35]:
#model construction using sklearn

from sklearn.ensemble import RandomForestRegressor
#import sklearn RF package

#creating an instance of RandomForestRegressor model with defined hyper-parameters
rfr = RandomForestRegressor(
    n_estimators=150,  # Default, can be changed later
    max_features=100, #sample 100 features (30 more features sample in snp-weather model)
    max_samples=0.65, #65% of samples used for each tr
    max_depth=None,  # Default
    min_samples_split=5,
    min_samples_leaf=1,
    n_jobs=-1,  # Use all CPU cores for faster training
    random_state=42)

In [36]:
print('fitting model...')
rfr.fit(X_train, y_train)
print('model is fit')

fitting model...
model is fit


In [37]:
#make prediction using model on val split
y_preds = rfr.predict(X_val)
y_preds.shape, y_preds[:10], y_val[:10]

((32507,),
 array([ 4.6557436 ,  8.87265357, 12.90220122, 13.26298252, 11.62097684,
        12.20573641,  6.89269748,  9.6670336 , 10.84209516,  7.78908991]),
 60271      3.476701
 59779      9.553867
 21123     13.233150
 158526    12.714523
 47749     13.420983
 2996      10.900502
 17978      5.957371
 54138     11.578752
 11456     12.139811
 2841       8.784822
 Name: Yield_Mg_ha, dtype: float64)

In [38]:
#metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

mse = mean_squared_error(y_val, y_preds)
mae = mean_absolute_error(y_val, y_preds)
r2 = r2_score(y_val, y_preds)

In [39]:
print(f'SNP-Weather-Management Random Forest Results: \nmean square error: {mse:.3f} | mean absolute error: {mae:.3f} | r squared {r2:.3f}')
#very similar if not same SNP-Weather model. appear adding management feature made little difference

#possible reasons: 
#mangament features are sparse in dataset + rarely deviate from default setting (too small for RF to detect)
#management features usually subtle + correlate with weather features (can't be detected by RF, might be detected by NN)
#RF is underfitting/ over regularised

SNP-Weather-Management Random Forest Results: 
mean square error: 3.061 | mean absolute error: 1.321 | r squared 0.684


In [40]:
y_preds = pd.Series(y_preds, index=y_val.index)

In [41]:
y_val_preds = pd.DataFrame({'y_preds': y_preds, 'y_val': y_val})

In [42]:
y_val_pred.to_csv('y_val_preds_snp_weather_management.csv', index=False)

In [44]:
#Calculate Gini Score to see feature importance in RF model
#Caution Gini score can be biased in unbalanced dataset
importances = rfr.feature_importances_

feature_importance_dict = dict(zip(X_train.columns, importances))
sorted_features = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

for feature, importance in sorted_features:
    print(f"{feature}: {importance:.4f}")

#weather features appear more important than snp or management
#max temp 4 weeks before planting is most informative. Could be period in which soil is prepared. high temp can effect soil moisture, microbes, etc
#some management features in top 10 (these a the numerical features with no missing values or n/a)
#most important cat management feature is Experiment Code (no missing values)
#snps features are very low in importance

#looks like management features with no missing values are used by model, ones with missing values and/or most are a default values is used less
#keep in dataset, these subtler effects and interaction maybe picked up by Neural network

T2M_MAX - Week -4: 0.0131
T2MDEW - Week 12: 0.0097
lat_Normalized: 0.0070
T2M_MIN - Week 12: 0.0062
T2M_MAX - Week 13: 0.0060
T2M_GDD - Week -4: 0.0056
T2M_GDD - Week 13: 0.0055
RH2M - Week -10: 0.0053
T2M_GDD - Week 12: 0.0051
lon_Normalized: 0.0047
Experiment_Code_Label_Normalized: 0.0047
T2M_GDD - Week 14: 0.0047
Env_Label_Normalized: 0.0046
RH2M - Week -9: 0.0045
T2MDEW - Week 11: 0.0044
T2M_GDD - Week -6: 0.0043
T2M_MIN - Week 13: 0.0043
T2M_MIN - Week -10: 0.0043
T2M_MAX - Week -9: 0.0043
WS2M - Week -7: 0.0042
T2M_MAX - Week -10: 0.0041
T2M_MAX - Week -8: 0.0041
T2M_MIN - Week 14: 0.0041
T2M_MAX - Week 17: 0.0040
T2M_MAX - Week -6: 0.0039
T2M_MAX - Week -7: 0.0039
T2M_GDD - Week -10: 0.0038
T2M_GDD - Week 11: 0.0037
WS2M - Week -9: 0.0037
T2M_MIN - Week -8: 0.0037
T2M_GDD - Week 15: 0.0036
RH2M - Week -4: 0.0036
T2MDEW - Week -10: 0.0035
Year_Normalized: 0.0035
T2M_MAX - Week 12: 0.0034
T2M_GDD - Week 8: 0.0034
T2M_GDD - Week -8: 0.0034
Pre-plant_tillage_method(s)_Label_Normaliz